In [30]:
import requests
from pathlib import Path
from datetime import datetime, timedelta
import pickle
import logging
import os

from utils import *
from tqdm.auto import tqdm
from bs4 import BeautifulSoup as bs

In [23]:
from meteostat import Point, Daily
from datetime import datetime

# Set time period
start = datetime(2015, 1, 1)
end = datetime(2024, 4, 12)

# Create Point for Vancouver, BC
location = Point(10.7905, 78.7047)

# Get daily data for 2018
data = Daily(location, start, end)
data = data.fetch()
data = data.reset_index()
data = data.fillna(method='ffill')
data

,time,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
0,2015-01-01,26.8,24.0,31.7,2.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-02,27.3,23.8,33.1,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-01-03,27.2,24.0,33.1,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-01-04,27.4,24.0,33.1,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-01-05,27.1,24.0,32.3,0.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3385,2024-04-08,32.8,26.0,41.0,0.0,NaN,136.0,9.2,NaN,1008.4,NaN
3386,2024-04-09,31.9,27.0,38.0,0.0,NaN,71.0,12.0,NaN,1009.5,NaN
3387,2024-04-10,31.9,27.0,38.5,0.0,NaN,61.0,13.1,NaN,1009.3,NaN
3388,2024-04-11,31.6,25.7,38.9,0.0,NaN,68.0,11.4,NaN,1009.5,NaN


In [24]:
temp = {}

def convert_dict(row):
    temp[row['time'].date()] = (row['tmin'], row['tmax'])

data.apply(convert_dict, axis=1)


0       None
1       None
2       None
3       None
4       None
        ... 
3385    None
3386    None
3387    None
3388    None
3389    None
Length: 3390, dtype: object

In [26]:
with open('../data/weather.pkl', 'wb') as f:
    pickle.dump(temp, f)

In [2]:
logger = logging.getLogger("my_custom_logger")
handler = logging.FileHandler("data_download.log")


logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

logger.addHandler(handler)


pricearrival = agmarknetOptions.price_arrival.Both
state = agmarknetOptions.state.__Select__


crop_name = dir(agmarknetOptions.commodities)
fol_path = Path('TN')


for crop in crop_name:
    
    commodity = eval(f'agmarknetOptions.commodities.{crop}')
    file_path = fol_path / f"{commodity[0].replace('/', '-')}.feather"
    from_dt = dt.date(2010, 1, 1)
    to_dt = dt.date(2024, 4, 11)

    if file_path.is_file():
        continue

    print(crop)
    try:

        df = []

        while from_dt < to_dt:
            cur_to_date = min(from_dt.replace(year=from_dt.year + 2), to_dt)
            print(from_dt, cur_to_date)
            response = agmarknet(from_dt.strftime('%Y-%m-%d'), cur_to_date.strftime('%Y-%m-%d'), pricearrival, commodity, state)
            
            if df:
                df += response[1:]
            else:
                df += response


            from_dt = cur_to_date + dt.timedelta(days=1)

        if df:
            df = pd.DataFrame(df[1:], columns=df[0])
            df.to_feather(file_path)
            logger.info(f'{crop}, {df.shape[0]}')
        else:
            with open(file_path, 'wb') as f:
                f.write("")
    
    except Exception as e:
        with open(file_path, 'wb') as f:
                f.write("error")
        logging.error(f'{crop}, {e}')

Amaranthus
2010-01-01 2012-01-01
2012-01-02 2014-01-02
2014-01-03 2016-01-03
2016-01-04 2018-01-04
2018-01-05 2020-01-05
2020-01-06 2022-01-06
2022-01-07 2024-01-07
2024-01-08 2024-04-11
Amphophalus
2010-01-01 2012-01-01
2012-01-02 2014-01-02
2014-01-03 2016-01-03
2016-01-04 2018-01-04
2018-01-05 2020-01-05
2020-01-06 2022-01-06
2022-01-07 2024-01-07
2024-01-08 2024-04-11
Apple
2010-01-01 2012-01-01


TypeError: a bytes-like object is required, not 'str'

In [63]:
## read rain fall from tn website

tn_weather = Path('../data/weather_org')

def get_dict_from_tn_html(soup: bs):

    sol = {}

    table = soup.select('#data_table')
    if not table: return sol

    table = table[0]
    cur_district = ''
    

    for row in table.select('tbody'):

        if not cur_district:
            cur_district = row.select('th')[2].text

        elif cur_district and len(row.select('th')) == 2:
            tds = row.select('td')
            sol[cur_district] = (float(tds[0].text), float(tds[1].text))
            cur_district = ''
    
    return sol


data = {}

for file in tqdm(os.listdir(tn_weather)):
    cur_file = pickle.load(open(tn_weather/file, 'rb'))
    data[file] = get_dict_from_tn_html(bs(cur_file))

  0%|          | 0/3390 [00:00<?, ?it/s]

In [91]:
data_in_list = []
index = []

for date in data:

    cur = {}

    for district in data[date]:
        cur[district] = data[date][district][0]

    data_in_list.append(cur)
    index.append(datetimredate)

df = pd.DataFrame(data_in_list, index=index).fillna(0)

TypeError: 'str' object cannot be interpreted as an integer

In [92]:
rainfall = df.apply(lambda x: x.mean().round(4), axis=1).to_dict()
with open('../data/rainfall.pkl', 'wb') as f:
    pickle.dump(rainfall, f)

In [45]:
cur_file = pickle.load(open(tn_weather/'2024-04-12', 'rb'))
soup = bs(cur_file)

In [61]:
get_dict_from_tn_html(soup)

{'Coimbatore': (5.0, 0.22),
 'Dindigul': (7.6, 0.76),
 'Kanniyakumari': (38.2, 1.47),
 'Pudukkottai': (4.2, 0.18),
 'Ramanathapuram': (19.2, 1.2),
 'Tenkasi': (73.6, 7.36),
 'The Nilgiris': (400.0, 13.79),
 'Thiruvarur': (17.8, 1.98),
 'Thoothukudi': (1.0, 0.05),
 'Tirunelveli': (415.0, 23.06),
 'Virudhunagar': (1.0, 0.08)}